In [2]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
from torch import nn
from torch.nn import functional as F
from tqdm import tqdm

In [3]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google'

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
device

'cuda'

In [ ]:
hadron_level_data = pd.read_csv("/content/drive/MyDrive/Pythiarecon/hadron_level_events.csv")
parton_level_data = pd.read_csv("/content/drive/MyDrive/Pythiarecon/parton_level_events.csv")

In [ ]:
hadron_level_data = hadron_level_data.drop(columns=["Unnamed: 91"]).dropna()
parton_level_data = parton_level_data.drop(columns=["Unnamed: 61"]).dropna()

In [ ]:
hadron_level_data

,18,1157.72,-0.111221,0.0185714,1.0796,0.199128,0.473307,0.941353,-0.352494,-0.0968177,...,0.26,0.27,0.28,0.29,0.30,0.31,0.32,0.33,0.34,0.35
0,20,1659.48,-0.153627,-0.081640,1.432670,-0.418220,-0.564112,0.729352,0.102893,0.150763,...,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,27,1444.55,0.189632,2.651150,0.799673,0.274388,2.369460,0.338421,0.503222,2.871830,...,2.45316,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,9,1139.93,-0.592578,-1.162090,1.493670,-0.148432,-1.545940,1.257400,-1.341100,-0.603664,...,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,8,1484.61,0.427927,1.816650,1.764040,-0.108304,1.534120,0.355921,-0.261452,1.659470,...,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,3,1422.98,-1.280210,1.933410,0.223974,-1.486320,2.663290,0.345800,-1.407430,2.329830,...,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8644,3,1009.51,-0.317243,0.470244,0.100079,-0.631486,0.913375,0.302620,-0.162838,1.163440,...,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8645,4,1117.03,0.730195,-2.605380,0.285309,2.064820,-2.542160,0.642341,0.573525,3.090320,...,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8646,5,1222.10,0.780720,-2.570710,0.397539,0.264381,-2.354080,0.438146,0.300481,-3.086570,...,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8647,4,1169.25,-0.443438,-2.277170,0.691763,-0.634910,-2.891280,0.552485,-0.719342,-2.701780,...,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
parton_level_data

,11,1157.72,-0.111221,0.0185714,1.01096,0.0372541,-0.457259,1.00512,-0.138702,0.522847,...,0.17,0.18,0.19,0.20,0.21,0.22,0.23,0.24,0.25,0.26
0,12,1659.48,-0.153627,-0.081640,4.643300,0.104736,-0.128314,3.684890,0.007839,-0.311173,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,10,1444.55,0.189632,2.651150,0.443718,0.629816,2.079270,1.884880,0.872034,2.704480,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,1139.93,-0.592578,-1.162090,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,1484.61,0.427927,1.816650,0.739495,1.101960,2.132700,1.831800,-0.111230,1.534950,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2,1422.98,-1.280210,1.933410,1.725520,-1.961120,1.739250,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8644,1,1009.51,-0.317243,0.470244,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8645,3,1117.03,0.730195,-2.605380,0.412928,0.234198,-2.907440,0.456253,1.376230,-2.197650,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8646,3,1222.10,0.780720,-2.570710,0.140600,0.698777,-2.125000,1.446580,1.202000,-2.693390,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8647,1,1169.25,-0.443438,-2.277170,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [1]:
x = hadron_level_data.values
y = parton_level_data.values

NameError: name 'hadron_level_data' is not defined

In [ ]:
scx = StandardScaler()
x = scx.fit_transform(x)

In [ ]:
x

In [ ]:
scy = StandardScaler()
y = scy.fit_transform(y)

In [ ]:
x

array([[ 1.17321938,  1.43477883, -0.20799801, ..., -0.03544477,
         0.01342283,  0.01643241],
       [ 2.12064269,  0.8313897 ,  0.2315107 , ..., -0.03544477,
         0.01342283,  0.01643241],
       [-0.31558869, -0.02379291, -0.77003071, ..., -0.03544477,
         0.01342283,  0.01643241],
       ...,
       [-0.85697344,  0.2068891 ,  0.98833956, ..., -0.03544477,
         0.01342283,  0.01643241],
       [-0.99231963,  0.05851932, -0.57907191, ..., -0.03544477,
         0.01342283,  0.01643241],
       [-0.85697344,  0.07171399, -0.52911327, ..., -0.03544477,
         0.01342283,  0.01643241]])

In [ ]:
class dataset(Dataset):
    def __init__(self, x, y):
        self.x = torch.tensor(x, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32)
        self.length = self.x.shape[0]

    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]

    def __len__(self):
        return self.length

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size=0.1, random_state=21, shuffle=False)

In [ ]:
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
Y_train = Y_train.astype('float32')
Y_test = Y_test.astype('float32')

In [ ]:
X_train = torch.from_numpy(X_train).to(device)
X_test = torch.from_numpy(X_test).to(device)
Y_train = torch.from_numpy(Y_train).to(device)
Y_test = torch.from_numpy(Y_test).to(device)

In [ ]:
X_train.dtype

torch.float32

In [ ]:
train_set = dataset(X_train, Y_train)
train_loader = DataLoader(train_set, batch_size=16,shuffle=False)

<ipython-input-13-8757b27423bd>:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.x = torch.tensor(x, dtype=torch.float32)
<ipython-input-13-8757b27423bd>:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.y = torch.tensor(y, dtype=torch.float32)


In [ ]:
test_set = dataset(X_test, Y_test)

<ipython-input-13-8757b27423bd>:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.x = torch.tensor(x, dtype=torch.float32)
<ipython-input-13-8757b27423bd>:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.y = torch.tensor(y, dtype=torch.float32)


In [ ]:
class Net(nn.Module):
    def __init__(self, input_shape):
        super(Net,self).__init__()
        self.fc1 = nn.Linear(input_shape, 64)
        self.fc2 = nn.Linear(64, 128)
        self.fc3 = nn.Linear(128, 256)
        self.fc4 = nn.Linear(256, 128)
        self.fc5 = nn.Linear(128, 61)

    def forward(self,x):
        x = self.fc1(x).relu()
        x = self.fc2(x).relu()
        x = self.fc3(x).relu()
        x = self.fc4(x).relu()
        x = self.fc5(x)
        return x

In [ ]:
lr = 0.0001
model = Net(input_shape=x.shape[1])
model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
# loss_fn = MSELoss()
loss_fn = nn.MSELoss()

In [ ]:
model.load_state_dict(torch.load("/content/drive/MyDrive/Pythiarecon/model.pth", weights_only=True))

<All keys matched successfully>

In [ ]:
# class EMDLoss(nn.Module):
#     def __init__(self):
#         super(EMDLoss, self).__init__()
#         self.R = 0.4

#     def forward(self, predictions, targets):
#         size_pred = predictions[0][0].item()
#         size_target = targets[0][0].item()
#         predictions = predictions[0][1:].reshape(20,3).cpu().detach()
#         predictions[predictions[:, 0] < 0, 0] = 0
#         targets = targets[0][1:].reshape(20,3).cpu()
#         emd_val= ef.emd.emd(predictions, targets, R=self.R, return_flow=False)
#         return torch.tensor([(size_pred-size_target)**2 + emd_val])

#     # def __init__(self):
#     #     super(EMDLoss, self).__init__()
#     #     self.weight = 1

#     # def forward(self, input, target):
#     #     # Compute the loss
#     #     loss = torch.mean(self.weight * (input - target) ** 2)
#     #     return loss

In [ ]:
epochs = 3000

In [ ]:
losses = []
accur = []
accur_test = []
for i in tqdm(range(epochs + 1)):
  for j,(x_train,y_train) in enumerate(train_loader):
    output = model(x_train)
    # print(output.shape)
    # print(y_train.shape)
    # print(output[0].shape)
    # print(output[0].shape)
    loss = loss_fn(output,y_train)


    # predicted = model(X_train)
    # predicted = predicted.reshape(-1).cpu().detach().numpy().round()
    # acc = (predicted == Y_train.cpu().numpy()).mean()

    # predicted_test = model(X_test)
    # predicted_test = predicted_test.reshape(-1).cpu().detach().numpy().round()
    # acc_test = (predicted_test == Y_test.cpu().numpy()).mean()

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # if i % 10 == 0:
  losses.append(loss.item())
  # accur.append(acc)
  # accur_test.append(acc_test)
  print("Epoch: {} \t Loss: {:.6f}".format(i,loss))
  # print("Epoch: {} \t Loss: {:.6f}\t Train Accuracy: {:.6f}\t Test Accuracy: {:.6f}".format(i,loss,acc, acc_test))

  0%|          | 1/3001 [00:00<45:16,  1.10it/s]

Epoch: 0 	 Loss: 0.167752


  0%|          | 2/3001 [00:01<45:04,  1.11it/s]

Epoch: 1 	 Loss: 0.166348


  0%|          | 3/3001 [00:02<44:09,  1.13it/s]

...

Epoch: 2998 	 Loss: 0.038144


100%|█████████▉| 3000/3001 [43:51<00:00,  1.12it/s]

Epoch: 2999 	 Loss: 0.040346


100%|██████████| 3001/3001 [43:52<00:00,  1.14it/s]

Epoch: 3000 	 Loss: 0.038392


In [ ]:
torch.save(model.state_dict(), '/content/drive/MyDrive/Pythiarecon/model_new.pth')

In [ ]:
predicted = model(X_test)

In [ ]:
predicted = scy.inverse_transform(predicted.detach().cpu().numpy())

In [ ]:
hadron_level = scx.inverse_transform(X_train.detach().cpu().numpy())

In [ ]:
hadron_level

array([[ 2.0000000e+01,  1.6594800e+03, -1.5362699e-01, ...,
         1.9145048e-09, -2.8880488e-11,  2.1256993e-11],
       [ 2.7000000e+01,  1.4445500e+03,  1.8963201e-01, ...,
         1.9145048e-09, -2.8880488e-11,  2.1256993e-11],
       [ 9.0000000e+00,  1.1399301e+03, -5.9257805e-01, ...,
         1.9145048e-09, -2.8880488e-11,  2.1256993e-11],
       ...,
       [ 1.2000000e+01,  1.2440601e+03, -4.4722900e-01, ...,
         1.9145048e-09, -2.8880488e-11,  2.1256993e-11],
       [ 4.0000000e+00,  1.1427700e+03, -2.1726899e-01, ...,
         1.9145048e-09, -2.8880488e-11,  2.1256993e-11],
       [ 9.0000000e+00,  1.4712700e+03,  4.3745002e-01, ...,
         1.9145048e-09, -2.8880488e-11,  2.1256993e-11]], dtype=float32)

In [ ]:
losses = np.array(losses)

In [ ]:
np.save("/content/drive/MyDrive/Pythiarecon/losses.npy", losses)

In [ ]:
num = 0

In [ ]:
predicted[num][1:].reshape(20,3)

array([[ 1.57917090e+03, -9.15718526e-02, -3.63428652e-01],
       [ 1.46344452e+01, -1.07878320e-01,  2.97439128e-01],
       [ 7.79153967e+00, -1.70613110e-01,  1.15418144e-01],
       [ 7.69062471e+00, -1.66948438e-01, -5.56872308e-01],
       [ 7.10284943e-03, -9.96207222e-02, -8.33377242e-01],
       [ 1.42533422e+00, -3.06877434e-01,  3.28714937e-01],
       [ 7.99401379e+00,  2.22430915e-01, -6.58364356e-01],
       [-4.85629767e-01,  2.26393923e-01,  3.15661848e-01],
       [ 2.33982921e+00, -1.10875696e-01,  3.55122775e-01],
       [-8.29269469e-01, -8.31894130e-02, -5.06968722e-02],
       [ 1.00286329e+00, -1.10874049e-01, -1.36076823e-01],
       [-7.09588379e-02, -1.79303847e-02,  5.95256016e-02],
       [-9.74664271e-01, -2.00887304e-02,  3.30156945e-02],
       [-2.99823377e-02,  1.10407192e-02, -2.98051033e-02],
       [ 1.88060757e-02,  4.14580898e-03, -2.28712577e-02],
       [-1.08720534e-01,  1.17854413e-03, -1.21601438e-03],
       [ 6.65265918e-02, -1.40871725e-03

In [ ]:
acc = scy.inverse_transform(Y_train.cpu().numpy())

In [ ]:
acc.shape

(7784, 61)

In [ ]:
acc[num][1:].reshape(20,3)

array([[ 1.6594800e+03, -1.5362701e-01, -8.1640095e-02],
       [ 4.6433005e+00,  1.0473600e-01, -1.2831399e-01],
       [ 3.6848900e+00,  7.8393398e-03, -3.1117299e-01],
       [ 1.0563301e+00,  5.9953398e-01, -1.9455302e-01],
       [ 1.0253004e+00,  3.2795701e-02, -8.4855902e-01],
       [ 6.3376600e-01, -6.4948201e-01,  3.2022402e-01],
       [ 9.1220117e-01,  2.6587698e-01, -6.1512303e-01],
       [ 5.9923702e-01,  2.3684900e-01,  2.2321302e-01],
       [ 3.6043200e+00, -2.2138600e-01,  6.7394698e-01],
       [ 1.2499599e+00, -1.4994700e-01,  3.7000801e-02],
       [ 5.8179599e-01, -3.6581600e-01,  4.2828301e-01],
       [ 2.2648199e+00,  2.2973101e-01,  5.4411799e-01],
       [ 3.8353276e-09,  7.2147940e-11, -7.0367170e-11],
       [ 2.1153125e-08,  1.7033312e-11, -8.9261237e-11],
       [ 5.2050630e-09,  6.1453523e-11,  1.1742382e-10],
       [-4.6975890e-09,  3.8900657e-11,  1.6942697e-13],
       [-4.5489817e-10,  2.0656714e-11, -1.8877508e-12],
       [-1.2217279e-09,  2.8144

In [ ]:
np.save("/content/drive/MyDrive/Pythiarecon/predicted.npy", predicted)

In [ ]:
np.save("/content/drive/MyDrive/Pythiarecon/acc.npy", acc)

In [ ]:
np.save("/content/drive/MyDrive/Pythiarecon/hadron_level.npy", hadron_level)